# Notacao IOB para NER (preprocess 2/2)

Notebook para incluir labels de entidades no esquema IOB para cada trecho de texto extraido em extract_text.ipynb

In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.model_selection import train_test_split
import math

## Parte 1 - Carregamento dos dados

In [2]:
# data = pd.read_csv("verif_data.csv")
data = pd.read_csv("verif_dataV1.csv")
data['labels'] = ""

In [3]:
data.columns

Index(['REF_ANOMES', 'DATA_DODF', 'NUM_DODF', 'PAGINA_DODF', 'TIPO_DODF',
       'ATO', 'COD_EMPRESA', 'EMPRESA_ATO', 'COD_MATRICULA_ATO',
       'COD_MATRICULA_SIGRH', 'CPF', 'NOME_ATO', 'NOME_SIGRH', 'CARGO',
       'CLASSE', 'PADRAO', 'QUADRO', 'PROCESSO', 'FUND_LEGAL', 'text',
       'labels'],
      dtype='object')

Colunas a serem utilizadas: 'ATO', 'EMPRESA_ATO', 'COD_MATRICULA_ATO', 'CPF', 'NOME_ATO', 'CARGO', 'CLASSE',                                   'PADRAO', 'QUADRO', 'PROCESSO', 'FUND_LEGAL'

Colunas descartadas: 'Unnamed: 0', 'REF_ANOMES', 'DATA_DODF', 'NUM_DODF', 'PAGINA_DODF',
       'TIPO_DODF', 'COD_EMPRESA', 'COD_MATRICULA_SIGRH', 'NOME_SIGRH'

In [4]:
data = data.drop(columns=[ 
    'REF_ANOMES', 
#     'DATA_DODF', 
#     'NUM_DODF', 
    'PAGINA_DODF', 
    'TIPO_DODF', 
    'COD_EMPRESA', 
    'COD_MATRICULA_SIGRH', 
    'NOME_SIGRH', 
    'CPF'
])

In [5]:
data.columns

Index(['DATA_DODF', 'NUM_DODF', 'ATO', 'EMPRESA_ATO', 'COD_MATRICULA_ATO',
       'NOME_ATO', 'CARGO', 'CLASSE', 'PADRAO', 'QUADRO', 'PROCESSO',
       'FUND_LEGAL', 'text', 'labels'],
      dtype='object')

In [6]:
print(data['FUND_LEGAL'][0], '\n\n')

# for row in range(len(data)):
#     for col in data.drop(columns=['text', 'labels']).columns:
#         if pd.notna(data[col][row]):
#             data.loc[row, col] = str(data.loc[row, col]).replace('º', 'o')

for row in range(len(data)):
    for col in data.drop(columns=['text', 'labels', 'DATA_DODF', 'NUM_DODF']).columns:
        if pd.notna(data.loc[row, col]):
            data.loc[row, col] = unicodedata.normalize('NFKD', str(data.loc[row, col])).encode('ascii', 'ignore').decode('utf8')

print(data['FUND_LEGAL'][0])

NOS TERMOS DO ARTIGO 3º, INCISOS I, II E III, E PARAGRAFO UNICO DA EMENDA CONSTITUCIONAL N.º 47 DE 05/07/2005, COMBINADO COM O ARTIGO 44 DA LEI COMPLEMENTAR N.º 769, DE 30/06/2008, COM A VANTAGEM PESSOAL PREVISTA NO ARTIGO 5º DA LEI Nº 4.584, DE 08/07/2011 


NOS TERMOS DO ARTIGO 3o, INCISOS I, II E III, E PARAGRAFO UNICO DA EMENDA CONSTITUCIONAL N.o 47 DE 05/07/2005, COMBINADO COM O ARTIGO 44 DA LEI COMPLEMENTAR N.o 769, DE 30/06/2008, COM A VANTAGEM PESSOAL PREVISTA NO ARTIGO 5o DA LEI No 4.584, DE 08/07/2011


## Parte 2 - Funcoes para identificar entidades e anotar no padrao IOB

In [7]:
def get_regex(data, col, idx):
    entity = str(data.loc[idx, col])
    re_entity = ""
    if col == "COD_MATRICULA_ATO":
        while entity[0] == '0':
            entity = entity[1:]
        re_entity += '[0oO]*?'
    for i in entity:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_entity += f"[{i.lower()}{i.upper()}]" + "[-,.\s\"]*?"
        elif i == '(' or i == ')':
            re_entity += '\\' + i + "[-,.\s\"]*?"
        else:
            re_entity += i + "[-,.\s\"]*?"
    return re_entity

In [8]:
def IOBify_text(text, entities):
    labels = ["O" for _ in range(len(text.split()))]
    word_start_position = []
    number_of_word = []
    for i in range(len(text)):
        if (i == 0):
            if (text[i] == ' '):
                continue
            else:
                number_of_word.append(len(word_start_position))
                word_start_position.append(i)
        elif text[i] != ' ' and text[i-1] == ' ':
            number_of_word.append(len(word_start_position))
            word_start_position.append(i)
    i = 0
    for entity in entities:
        entity_begin = entity_end = -1
        # Find initial position of entity
        if entity[0] in word_start_position:
            entity_begin = word_start_position.index(entity[0])
        # Find final position of entity
        for pos, idx in zip(word_start_position, number_of_word):
            if pos > entity[1]:
                entity_end = idx-1
                break
        if entity_end == -1:
            entity_end = number_of_word[-1]
            
        if entity_begin != -1:
            for i in range(entity_begin, entity_end+1):
                if i==entity_begin:
                    labels[i] = "B-"+entity[2]
                else:
                    labels[i] = "I-"+entity[2]
    return labels
                
def find_entities(data, idx):
    list_entities = []
    for col in data.drop(columns=['text', 'labels', 'DATA_DODF', 'NUM_DODF']).columns:
        if pd.notna(data[col][idx]):
            re_entity = get_regex(data, col, idx)
            aux = re.search(re_entity, data['text'][idx])
            if aux:
                found_entities[col]  += 1
                list_entities.append([aux.span()[0], aux.span()[1], col])
            else:
                missed_entities[col][0] += 1
                missed_entities[col][1].append(idx)
    return list_entities


## Parte 3 - Identificando entidades e anotando-as

In [9]:
found_entities  = {col: 0 for col in data.drop(columns=['text', 'labels', 'DATA_DODF', 'NUM_DODF']).columns}
missed_entities = {col: [0, []] for col in data.drop(columns=['text', 'labels', 'DATA_DODF', 'NUM_DODF']).columns}

for row in range(len(data)):
    if pd.notna(data.loc[row, 'text']):
        entities = find_entities(data, row)
        entities.sort()
        labels = IOBify_text(data.loc[row, 'text'], entities)
        s = ""
        for l in labels:
            s += l + ' '
        data.loc[row, 'labels'] = s

for col in found_entities:
    print(f"Found {col}: {found_entities[col]}", f"\t\t Missed {col}: {missed_entities[col][0]}")

Found ATO: 5511 		 Missed ATO: 5
Found EMPRESA_ATO: 2476 		 Missed EMPRESA_ATO: 3040
Found COD_MATRICULA_ATO: 5504 		 Missed COD_MATRICULA_ATO: 12
Found NOME_ATO: 5512 		 Missed NOME_ATO: 4
Found CARGO: 5508 		 Missed CARGO: 7
Found CLASSE: 2710 		 Missed CLASSE: 40
Found PADRAO: 4956 		 Missed PADRAO: 5
Found QUADRO: 4992 		 Missed QUADRO: 45
Found PROCESSO: 5182 		 Missed PROCESSO: 36
Found FUND_LEGAL: 4830 		 Missed FUND_LEGAL: 685


## Parte 4 - Corrigindo entidades nao identificadas

In [10]:
# Carregamento dos dados originais anotados
data_og = pd.read_csv('TCDF_data/Atos_Aposentadoria_validados.csv')

### Correcao manual de trechos de texto identificados erroneamente

Trechos de texto que nao condizem com as entidades anotadas sao corrigidos aqui

In [11]:
# Matricula(DONE) | NOME(DONE) | CARGO(DONE) | CLASSE(DONE) | PADRAO(DONE) | QUADRO(DONE) | PROCESSO()

# Fix row 138
data.loc[138, 'text'] = "CONCEDER aposentadoria voluntaria com proventos integrais a servidora a seguir nominada: MARIA AUXILIADORA PE- REIRA, matricula 101.008-5, processo SEI n 00070- 00011305/2018-23, no cargo de Au- xiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, com fundamento no Art. 6, incisos I, II, III e IV, da EC n 41/ 2003, combinado com o artigo 2 da EC n 47/2005, c/c a Lei Complementar no 769/08."

# Fix row 481
data.loc[481, 'text'] = "CONCEDER APOSENTADORIA a IZAAC NEWTON DA SILVA, matricula 38.950-1, no Cargo de Professor de Educacao Basica, Padrao 22, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080-00033434/2017-54."

# Fix row 506
data.loc[506, 'text'] = "CONCEDER aposentadoria voluntaria com proventos integrais ao servidor a seguir nominado: AMISAEL GONCALVES BI- NACETT, matricula 100.453-0, processo SEI n 00070-00013097/2018-05, no cargo de Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, com fundamento no Art. 3, incisos I, II, III e paragrafo unico da EC no 47/2005, c/c a LC no 769/2008."
data.loc[506, 'PROCESSO'] = "00070-00013097/2018-05"
data.loc[506, 'CARGO'] = "Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria"

# Fix row 537
data.loc[537, 'text'] = "CONCEDER APOSENTADORIA a VERA LUCIA FERREIRA DE SOUSA, matricula 40.787-9, no Cargo de Agente de Gestao Educacional/Servicos Gerais, Nivel 10, Padrao 02, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 3o da Emenda Constitucional no 47, de 06 de julho de 2005 e o Paragrafo unico do mesmo artigo. Processo 0 0 0 8 0 - 0 0 0 0 3 4 3 2 / 2 0 1 8 - 11 ."
data.loc[537, 'PROCESSO'] = "0 0 0 8 0 - 0 0 0 0 3 4 3 2 / 2 0 1 8 - 11"

#Fix row 571
data.loc[571, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a MARIA DE LOURDES SOUSA, matricula no 0143048-3, na Carreira de Assistencia Publica a Saude, no Cargo de Tecnico em Saude - TEC. LAB. HEMAT. E HEMOT, Primeira Classe, Padrao II, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: ADMC. Processo no 00060- 00247085/2017-21."

# Fix row 2763
data.loc[2763, 'text'] = "APOSENTAR MARIA ESTER BATISTA DE MEDEIROS, matricula 204.859-0, no Cargo de Professor de Educacao Basica, Padrao 19, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012. Processo 00040-00015628/2019-89."

# Fix row 4851
data.loc[4851, 'text'] = "CONCEDER APOSENTADORIA a KATIA MARIA ALMEIDA, matricula 49.118-7, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005, e com o artigo 40,  5o, da Constituicao da Republica Federativa do Brasil. Processo 00080-00029812/2018-86"

# Fix row 5086
data.loc[5086, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com artigo 44 da Lei Complementar no 769, de 30/06/2008, bem como as vantagens do art. 5 da Lei 4.584/2011, a LUIZ CARLOS CHAVES E SILVA, matricula no 1.400.743-6, na Carreira de Auditoria de Atividades Ur- banas, no Cargo de AUDITOR DE ATIVIDADES URBANAS, Classe Especial, Padrao V, do Quadro de Pessoal do Distrito Federal. Lotacao: ADMC. Processo: 00060- 00256728/2017-27"

# Fix row 5512
data.loc[5512, 'text'] = "APOSENTAR, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a MARIA DE LOURDES SOUSA, matricula no 0143048-3, na Carreira de Assistencia Publica a Saude, no Cargo de Tecnico em Saude - TEC. LAB. HEMAT. E HEMOT, Primeira Classe, Padrao II, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: ADMC. Processo no 00060- 00247085/2017-21."


## Correcao manual das entidades de ATO

In [12]:
# Fix row 403
data.loc[403, 'ATO'] = "APOSENTADORIA VOLUNTARIA"

# Fix row 982
data.loc[982, 'ATO'] = "APOSENTADORIA VOLUNTARIA"

# Fix row 3846
data.loc[3846, 'ATO'] = "APOSENTADORIA VOLUNTARIA"

# Fix row 3853
data.loc[3853, 'ATO'] = "APOSENTADORIA VOLUNTARIA"

### Correcao manual das entidades de MATRICULA

utilizando as matriculas SIGRH quando matriculas ATO nao sao encontradas

In [13]:
# Fix row 116
data.loc[116, 'COD_MATRICULA_ATO'] = "100.723-8"

# Fix row 138
data.loc[138, 'COD_MATRICULA_ATO'] = data_og.loc[138, 'COD_MATRICULA_SIGRH']

# Fix row 236
data.loc[236, 'COD_MATRICULA_ATO'] = "100.997-4"

# Fix row 422
data.loc[422, 'COD_MATRICULA_ATO'] = data_og.loc[422, 'COD_MATRICULA_SIGRH']

# Fix row 506
data.loc[506, 'COD_MATRICULA_ATO'] = data_og.loc[506, 'COD_MATRICULA_SIGRH']

# Fix row 537
data.loc[537, 'COD_MATRICULA_ATO'] = data_og.loc[537, 'COD_MATRICULA_SIGRH']

# Fix row 719
data.loc[719, 'COD_MATRICULA_ATO'] = data_og.loc[719, 'COD_MATRICULA_SIGRH']

# Fix row 761
data.loc[761, 'COD_MATRICULA_ATO'] = data_og.loc[761, 'COD_MATRICULA_SIGRH']

# Fix row 3610
data.loc[3610, 'COD_MATRICULA_ATO'] = data_og.loc[3610, 'COD_MATRICULA_SIGRH']

### Correcao manual das entidades de NOME

Diferenca no processamento de acentos e apostrofos entre o DODFminer e as anotacoes

In [14]:
# Nenhuma correcao necessaria

### Correcao manual das entidades de CARGO

Algumas anotacoes erroneas (apresentadas abaixo)

In [15]:
# Fix row 719
data.loc[719, 'CARGO'] = "agente de transito"

# Fix row 4141
data.loc[4141, 'CARGO'] = "Gestor em Politicas Publicas e Gestao Governamental"

### Correcao manual das entidades de CLASSE

Em grande parte diferenca entre numeros ordinais e suas escritas por extenso (1a classe == primeira classe)

In [16]:
for i in missed_entities['CLASSE'][1]:
    a = data.iloc[i]['CLASSE']
    a = a.split()
    s = ""
    if a[0].lower() == 'primeira':
        s = "1a Classe"
    elif a[0].lower() == 'segunda':
        s = "2a Classe"
    elif a[0].lower() == 'terceira':
        s = "3a Classe"
    elif a[0].lower() == 'quarta':
        s = "4a Classe"
    if s:
        data.loc[i, 'CLASSE'] = s
        
# Fix row 506
data.loc[506, 'CLASSE'] = "Classe Unica"

# Fix row 571
data.loc[571, 'CLASSE'] = "Primeira Classe"

# Fix row 947
data.loc[947, 'CLASSE'] = "Classe Unica"

# Fix row 5512
data.loc[5512, 'CLASSE'] = "Primeira Classe"


### Correcao manual das entidades de PADRAO

Entidades da classe PADRAO foram anotadas erroneamente (casos abaixo)

In [17]:
# # Fix row 3199
# data.loc[3199, 'PADRAO'] = "Padrao 15"

# # Fix row 4449
# data.loc[4449, 'PADRAO'] = "Padrao 25"

### Correcao manual das entidades de QUADRO

Ainda nao sei qual o erro

In [18]:
for i in missed_entities['QUADRO'][1]:
    if data.loc[i, 'QUADRO'] == 'QUADRO DE PESSOAL DO SERVICO DE LIMPEZA URBANA':
        data.loc[i, 'QUADRO'] = "QP/SLU"
        
# Fix row 3927 (DODFminer pulou parte do texto do DODF)
data.loc[3927, 'QUADRO'] = "Quadro de Pessoal do Governo do Distrito"

# Fix row 104 (DODFminer pulou parte do texto do DODF)
data.loc[104, 'QUADRO'] = "Quadro de"

### Correcao manual das entidades de EMPRESA_ATO

Algumas anotacoes nao aparecem no DODF

### Correcao manual das entidades de FUND_LEGAL

Tentativa de usar o comeco e o final das entidades de FUND_LEGAL (3 primeiras e tres ultimas palavras

Para as entidades nao encontradas, correcao manual

#### Utilizando as 3 primeiras e 3 ultimas palavras da entidade FUND_LEGAL anotada

In [19]:
# Utilizando as tres primeiras e tres ultimas palavras da entidade anotada (ao inves de a entidade completa) com regex

print(data.loc[290, 'FUND_LEGAL'])
for sample in missed_entities['FUND_LEGAL'][1]:
    s = data.loc[sample, 'FUND_LEGAL'].split()
    beg = ' '.join(s[:4])
    end = ' '.join(s[-3:len(s)]).replace(' NAo ', ' No ')
        
    # Criando regex para a entidade FUND_LEGAL (utilizando as primeiras 3 e as ultimas 3 palavras de data.loc[sample, 'FUND_LEGAL'])
    re_FUND_LEGAL = ""
    for i in beg:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_FUND_LEGAL += f"[{i.lower()}{i.upper()}]" + "[-,.\s]*?"
        else:
            re_FUND_LEGAL += i + "[-,.\s]*?"
    re_FUND_LEGAL += '.*?'
    for i in end:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_FUND_LEGAL += f"[{i.lower()}{i.upper()}]" + "[-,.\s]*?"
        else:
            re_FUND_LEGAL += i + "[-,.\s]*?"
            
    # Busca do regex no trecho de texto que deve conter esta entidade
    result = re.search(re_FUND_LEGAL, data.loc[sample, 'text'])
    if result:
        if abs(len(data.loc[sample, 'text'][result.span()[0]:result.span()[1]].split()) - len(data.loc[sample, 'FUND_LEGAL'].split())) < 3:
            data.loc[sample, 'FUND_LEGAL'] = result[0]
            
print(data.loc[290, 'FUND_LEGAL'])


NOS TERMOS DO ARTIGO 40,  1o, INCISO III, ALINEA AE  3o, 5o,8o E 17, DA CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, NA REDACAO DADA PELA EMENDA CONSTITUCIONAL No 41, DE 31 DE DEZEMBRO DE 2003, E ARTIGOS 46 E 51 DA LEI COMPLEMENTAR No 769, DE 01 DE JULHO DE 2008
nos termos do artigo 40,  1o, inciso III, alinea "a"e 3o, 5o,8o e 17, da Constituicao da Republica Federativa do Brasil, na redacao dada pelaEmenda Constitucional no 41, de 31 de dezembro de 2003, e artigos 46 e 51 da LeiComplementar no 769, de 01 de julho de 2008


#### correcao manual

In [20]:
# Fix row 947
data.loc[947, 'FUND_LEGAL'] = 'nos termos do artigo 40,  1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 41, de 19/12/2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 19/12/2003, incluido pela Emenda Constitucional no. 70, de 29/03/2012, com a vantagem pessoal prevista no artigo 5o da Lei no 4.584, de 08 / 0 7 / 2 0 11'

# Fix row 4023
data.loc[4023, 'FUND_LEGAL'] = 'fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 4091
data.loc[4091, 'FUND_LEGAL'] = 'nos termos do artigo 40, 1o, inciso I, e  3o, 8o e 17, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41/2003, e artigos 46 e 51 da Lei Complementar 769/2008, de acordo com o Laudo Medico no 032/2017 e Laudo de Junta Medica Oficial de Recurso n 019/2017'

# Fix row 4265
data.loc[4265, 'FUND_LEGAL'] = 'nos termos do artigo 6o da Emenda Constitucional no 41/2003, c/c o artigo 2o da Emenda Constitucional no 41/2003'

# Fix row 4351
data.loc[4351, 'FUND_LEGAL'] = 'fundamentada no art. 3o, incisos I, II e III, e paragrafo unico, da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98'

# Fix row 4352
data.loc[4352, 'FUND_LEGAL'] = 'fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista nos artigos 1o e 7o da Lei no 1.004/96, combinados com o artigo 4o da Lei no 1.141/96, e com o artigo 4o, Paragrafo unico, da Lei no 1.864/98'

# Fix row 4353
data.loc[4353, 'FUND_LEGAL'] = 'fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 1o e 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, e com o art. 4o, paragrafo unico, da Lei no 1.864/98'

# Fix row 4365
data.loc[4365, 'FUND_LEGAL'] = 'nos termos do artigo 40,  1o, inciso I, in fine, e  3o, 8o e 17, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 41, de 19/12/2003, e artigos 18,  5o, 46 e 51 da Lei Complementar no 769, de 30/06/2008'

# Fix row 4368
data.loc[4368, 'FUND_LEGAL'] = 'fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05'

# Fix row 4383
data.loc[4383, 'FUND_LEGAL'] = 'fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98'

# Fix row 4388
data.loc[4388, 'FUND_LEGAL'] = 'fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05'

# Fix row 4416
data.loc[4416, 'FUND_LEGAL'] = 'na redacao dada pela Emenda Constitucional no 41/2003, e artigos 18,  5o, 46 e 51 da Lei Complementar no 769/08, de acordo com o Laudo Medico no 042/2017'

# Fix row 4426
data.loc[4426, 'FUND_LEGAL'] = 'nos termos do artigo 40,  1o, inciso I, in fine, e  3o, 8o e 17, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 41, de 19/12/2003, e artigos 18,  5o, 46 e 51 da Lei Complementar no 769, de 30/06/2008'

# Fix row 4990
data.loc[4990, 'FUND_LEGAL'] = 'nos termos do artigo 6o SECRETARIA DE ESTADO DE EDUCACAO da Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005'


### Correcao manual de trechos das entidades de PROCESSO

In [21]:
# Fix row 124
data.loc[124, 'text'] = 'A DIRETORA DO DEPARTAMENTO DE GESTAO DE PESSOAS, DA POLICIA CIVIL DO DISTRITO FEDERAL, no uso de suas atribuicoes legais e tendo em vista a delegacao de competencia que lhe foi conferida pelo artigo 1o da Portaria no 3, de 11 de janeiro de 2012, e, ainda, o constante no processo 052.001.852/2013, RESOLVE: CONCEDER apo- sentadoria a WILSON FRANCISCO DA SILVA, matricula no 58.678-1, no cargo efetivo de Agente Policial de Custodia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 6o da Emenda Constitucional no 41/2003 c/c o artigo 2o da Emenda Constitucional no 47/2005.'

# fix row 973
data.loc[973, 'text'] = "O PRESIDENTE EM EXERCICIO DO TRIBUNAL DE CONTAS DO DISTRITO FE- DERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no29.644/2017-e, RESOLVE: CONCEDER aposentadoria voluntaria a servidora IVONETE PETRONILO DE AQUINO, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula 988-1, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no artigo 1o da Lei no 1.004/96, combinado com os artigos 4o da Lei no 1.141/96 e 4o, paragrafo unico, da Lei no 1.864/98."

# Fix row 1059
data.loc[1059, 'text'] = "O PRESIDENTE EM EXERCICIO DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 18.329/2018-e, resolve: Conceder aposentadoria voluntaria ao servidor EDUARDO MARIA DE MACEDO FRANCA, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula 276-3, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98."

# Fix row 1070
data.loc[1070, 'text'] = "O DIRETOR DO DEPARTAMENTO DE GESTAO DE PESSOAS, DA POLICIA CIVIL DO DISTRITO FEDERAL, no uso de suas atribuicoes legais e tendo em vista a delegacao de competencia que lhe foi conferida pelo artigo 1o da Portaria no 3, de 11 de janeiro de 2012, e, ainda, o constante no processo 00052- 00022842/2018-17, resolve: CONCEDER aposentadoria a LEONIDAS PEREIRA DE ALMEIDA, matricula no 57.863-0, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA."

# Fix row 1227
data.loc[1227, 'text'] = "A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 1.499/2019-e, resolve: CONCEDER aposentadoria voluntaria a servidora SANDRA DE ANDRADE PACHECO, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula no 1270-0, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05."

# Fix row 1261
data.loc[1261, 'text'] = "O PRESIDENTE EM EXERCICIO DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 906/2019-e, resolve: CONCEDER aposentadoria voluntaria a servidora JOSEFA DA SILVA RIBEIRO DE AVILA, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula no 1021-9, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 1o e 7o da Lei no 1.004/96, c/c o art. 4o da Lei no 1.141/96, e com o art. 4o, paragrafo unico, da Lei no 1.864/98."

# Fix row 1398
data.loc[1398, 'text'] = 'O DIRETOR DO DEPARTAMENTO DE GESTAO DE PESSOAS DA POLICIA CIVIL DO DISTRITO FEDERAL, no uso de suas atribuicoes legais e tendo em vista a delegacao de competencia que lhe foi conferida pelo artigo 1o, inciso IV, da Portaria no 25, de 18 de junho de 2018, e, ainda, o constante no processo 00052-00002900/2019-77, resolve: CONCEDER aposentadoria a FRANCISCO FIDELMAN ALVES DE LIMA, matricula no 46.846-0, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA RE- PUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003.'

# Fix row 1419
data.loc[1419, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 21.320/2018-e, resolve: Conceder aposentadoria voluntaria ao servidor JOAO ADRIANO DE CARVALHO, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula 500-2, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 1606
data.loc[1606, 'text'] = ' PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 5338/2019-e, resolve: Conceder aposentadoria voluntaria ao servidor EDNALDO NOGUEIRA AMARAL, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 1199-1, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 1607
data.loc[1607, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 20269/2018-e, resolve: Conceder aposentadoria voluntaria ao servidor ITAMAR LINO DE OLIVEIRA, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 1101- 1, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 1o e 7o da Lei no 1.004/96, c/c o art. 4o da Lei no 1.141/96, e com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 1611
data.loc[1611, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 10471/2017-e, resolve: CONCEDER aposentadoria voluntaria ao servidor CLERIO BORBA BRASIL, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula no 414-6, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 1716
data.loc[1716, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 5.419/2019-e, resolve: CONCEDER aposentadoria voluntaria a servidora LUCIANA DE ALBUQUERQUE MELLO SEIXAS, Auditora de Controle Externo, Classe Especial, Padrao 63, matricula no 546-1, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 2000
data.loc[2000, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 8310/2019-e, resolve: Conceder aposentadoria voluntaria a servidora ROSELI RAPOSO, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula 1030-8, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 2101
data.loc[2101, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 36.696/2018-e, resolve: CONCEDER aposentadoria voluntaria a JOSIMAR DIAS MATOS, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 836-2, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 2272
data.loc[2272, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 11.671/2019-e, resolve: CONCEDER aposentadoria voluntaria ao servidor GIVALDO ANTONIO BATISTA DA CUNHA, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 990-3, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 2299
data.loc[2299, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 36.858/2018-e, resolve: CONCEDER aposentadoria voluntaria a servidora CELINA PEREIRA DE SOUZA, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula no 1042-1, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no artigo 1o da Lei no 1.004/96, combinado com os artigos 4o da Lei no 1.141/96 e 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 2393
data.loc[2393, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 18.876/2018-e, resolve: CONCEDER aposentadoria voluntaria a servidora ELIZABETH DAS GRACAS CALDEIRA BRANT OLIVEIRA, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula no 757-9, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no artigo 1o da Lei no 1.004/96, combinado com os artigos 4o da Lei no 1.141/96 e 4o, paragrafo unico, da Lei 1.864/98.'

# Fix row 2444
data.loc[2444, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 2690/2019-e, resolve: CONCEDER aposentadoria voluntaria a servidora ROSILAINY DA FONSECA SIQUEIRA, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula 1006-5, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 2574
data.loc[2574, 'text'] = 'O PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, Em Exercicio, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 9074/2019-e, resolve: CONCEDER aposentadoria voluntaria ao servidor PEDRO JOSE MARTINS SALGADO, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula 932-6, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista nos artigos 1o e 7o da Lei no 1.004/96, combinados com o artigo 4o da Lei no 1.141/96, e com o artigo 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 3049
data.loc[3049, 'text'] = 'O PRESIDENTE EM EXERCICIO DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 8639/2019-e, resolve: CONCEDER aposentadoria voluntaria ao servidor ULISSES APARECIDO RIBEIRO, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula 964-4, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 3070
data.loc[3070, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 6236/2017-e, resolve: CONCEDER aposentadoria voluntaria ao servidor HENRIQUE MATTEI, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula no 526-6, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 4023
data.loc[4023, 'text'] = 'A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 11.715/2018-e, resolve: Conceder aposentadoria voluntaria a servidora TELMA FONTINELE QUEIROZ, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula 1029-4, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 4300
data.loc[4300, 'text'] = 'A DIRETORA DO DEPARTAMENTO DE GESTAO DE PESSOAS, DA POLICIA CIVIL DO DISTRITO FEDERAL, no uso de suas atribuicoes legais e tendo em vista a delegacao de competencia que lhe foi conferida pelo artigo 1o da Portaria no 3, de 11 de janeiro de 2012, e, ainda, o constante no processo 00052-00020632/2018-94, resolve: CONCEDER aposentadoria a NAGELA CRISTIEN DO MONT, matricula no 43.492-2, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da Constituicao da Republica Federativa do Brasil, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003.'

# Fix row 4351
data.loc[4351, 'text'] = "A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inciso III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 24081/17-e, resolve: Conceder aposentadoria voluntaria ao servidor MARIO NUNES ATAIDES, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 901-6, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico, da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98."

# Fix row 4352
data.loc[4352, 'text'] = "A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inciso III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 40.249/2017-e, resolve: Conceder aposentadoria voluntaria ao servidor PAULO CEZAR CARNEIRO, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula 316-6, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista nos artigos 1o e 7o da Lei no 1.004/96, combinados com o artigo 4o da Lei no 1.141/96, e com o artigo 4o, Paragrafo unico, da Lei no 1.864/98."

# Fix row 4353
data.loc[4353, 'text'] = "A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inciso III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 29.420/2014-e, resolve: Conceder aposentadoria voluntaria a servidora GEUSA SANTANA DA SILVA, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula 1122-3, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 1o e 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, e com o art. 4o, paragrafo unico, da Lei no 1.864/98."

# Fix row 4368
data.loc[4368, 'text'] = "A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 36.629/2018-e, resolve: Conceder aposentadoria voluntaria ao servidor FRANCISCO SOARES DA SILVA, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 781-1, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05."

# Fix row 4383
data.loc[4383, 'text'] = "A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 31.538/2018-e, resolve: Conceder aposentadoria voluntaria a servidora GIRLENE MOREIRA DA SILVA, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula 790-1, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98."

# Fix row 4388
data.loc[4388, 'text'] = "A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 16.879/2017-e, resolve: Conceder aposentadoria voluntaria ao servidor FRANCISCO JOSCELY TEIXEIRA ALBUQUERQUE, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula no 521-5, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05."

# Fix row 4468
data.loc[4468, 'text'] = "A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 4050/2018-e, resolve: CONCEDER aposentadoria por invalidez permanente a servidora LEILA MAGDA DE MELO, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula no 1103-7, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, com fundamento no artigo 40,  1o, inciso I, in fine, da CRFB, na redacao dada pela Emenda Constitucional no 41/03, combinado com o artigo 6o-A da Emenda Constitucional no 41/03, incluido pela Emenda Constitucional no 70/12, e o art. 18,  5o, da Lei Complementar no 769/08, com a vantagem pessoal prevista nos artigos 1o e 7o da Lei no 1.004/96, combinados com o artigo 4o da Lei no 1.141/96, e com o artigo 4o, paragrafo unico, da Lei no 1.864/98."

# Fix row 4796
data.loc[4796, 'PROCESSO'] = "00060-00054308/2018-99"

# Fix row 5002
data.loc[5002, 'text'] = "A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 23.292/2018-e, RESOLVE: CONCEDER aposentadoria voluntaria ao servidor WILLIAM DA COSTA, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula 975-0, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no artigo 7o da Lei no 1.004/96, combinado com os artigos 4o da Lei no 1.141/96 e 4o, paragrafo unico, da Lei no 1.864/98."

# Fix row 5030
data.loc[5030, 'text'] = "A DIRETORA DO DEPARTAMENTO DE GESTAO DE PESSOAS, DA POLICIA CIVIL DO DISTRITO FEDERAL, no uso de suas atribuicoes legais e tendo em vista a delegacao de competencia que lhe foi conferida pelo artigo 1o da Portaria no 3, de 11 de janeiro de 2012, e, ainda, o constante no processo 00052.00011934/2018-71 RESOLVE: AP O S E N TA R a servidora KLEYTAMAR DE ARAUJO, matricula no 57.594-1, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 40o, paragrafo 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41/2003, c/c artigo 6o-A da Emenda Cons- titucional no 41/2003, incluido pela Emenda Constitucional no 70/2012 e o artigo 186, inciso I, da Lei no 8.112/90."

# Fix row 5142
data.loc[5142, 'PROCESSO'] = "00060-00034069/2018-51"

# Fix row 5170
data.loc[5170, 'text'] = "A PRESIDENTE DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 24.639/2018-e, RESOLVE: CONCEDER aposentadoria voluntaria a servidora OLGA LEITE DA SILVA, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula no 1109-6, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05."

## Parte 5 - Reanotando entidades corrigidas

Similar a Parte 3, mas agora utilizando os dados corrigidos na Parte 4

In [22]:
found_entities  = {col: 0 for col in data.drop(columns=['text', 'labels', 'DATA_DODF', 'NUM_DODF']).columns}
missed_entities = {col: [0, []] for col in data.drop(columns=['text', 'labels', 'DATA_DODF', 'NUM_DODF']).columns}
data['labels'] = ""
for row in range(len(data)):
    if pd.notna(data['text'][row]):
        entities = find_entities(data, row)
        entities.sort()
        labels = IOBify_text(data['text'][row], entities)
        s = ""
        for l in labels:
            s += l + ' '
        data['labels'][row] = s
    
for col in found_entities:
    print(f"Found {col}: {found_entities[col]}", f"\t\t Missed {col}: {missed_entities[col][0]}")

<ipython-input-22-996c51c1db21>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['labels'][row] = s


Found ATO: 5515 		 Missed ATO: 1
Found EMPRESA_ATO: 2506 		 Missed EMPRESA_ATO: 3010
Found COD_MATRICULA_ATO: 5516 		 Missed COD_MATRICULA_ATO: 0
Found NOME_ATO: 5516 		 Missed NOME_ATO: 0
Found CARGO: 5514 		 Missed CARGO: 1
Found CLASSE: 2749 		 Missed CLASSE: 1
Found PADRAO: 4960 		 Missed PADRAO: 1
Found QUADRO: 5036 		 Missed QUADRO: 1
Found PROCESSO: 5217 		 Missed PROCESSO: 1
Found FUND_LEGAL: 5515 		 Missed FUND_LEGAL: 0


## Final - salvando os dados preprocessados

### Salvando no formato CoNLL03 

Salva os trechos de texto e labels, separando em conjuntos "nao rotulado" e "rotulado" para teste com active learning

In [23]:
x = []
y = []
t = []

# Separa conjuntos de dados de forma cronológica
def func(a):
    a = a.split('/')
    return a[2], a[1], a[0]
for row in data.index:
    if pd.notna(data['text'][row]) and pd.notna(data['labels'][row]):
        x.append(data.loc[row, 'text'])
        y.append(data.loc[row, 'labels'])
        t.append(' '.join((lambda x: x[::-1])(data.loc[row, 'DATA_DODF'].split('/'))))
idx = np.argsort(t)
test_split_idx = math.floor(len(idx)*0.7)
t1 = [-1 for _ in range(len(t))]
for i, j in enumerate(idx):
    t1[i] = t[j]
x_train = [x[i] for i in idx[:test_split_idx]]
y_train = [y[i] for i in idx[:test_split_idx]]
x_test = [x[i] for i in idx[test_split_idx:]]
y_test = [y[i] for i in idx[test_split_idx:]]

initial_labeled_set_idx = math.floor(len(x_train)*0.99)
labeled_x = x_train[initial_labeled_set_idx:]
labeled_y = y_train[initial_labeled_set_idx:]
unlabeled_x = x_train[:initial_labeled_set_idx]
unlabeled_y = y_train[:initial_labeled_set_idx]
print("Tamanho do conjunto de treinamento")
print("unlabeled set size:", len(unlabeled_x), "labeled set size:",  len(labeled_x))
print("\nTamanho do conjunto de teste")
print("test set size:", len(x_test))

Tamanho do conjunto de treinamento
unlabeled set size: 3822 labeled set size: 39

Tamanho do conjunto de teste
test set size: 1655


In [24]:
def save_conll(x, y, nome_arquivo):
    ptr = open(nome_arquivo, 'w')
    for i in range(len(x)):
        text = x[i]
        label = y[i]
        assert len(text.split()) == len(label.split()), f"Erro em {text}"
        for word, label in zip(text.split(), label.split()):
            ptr.write(word + ' ' + label + '\n')
        ptr.write('\n')
save_conll(labeled_x,   labeled_y,   "labeled_set.txt"  )
save_conll(unlabeled_x, unlabeled_y, "unlabeled_set.txt")
save_conll(x_test,      y_test,      "test_set.txt"     )
save_conll(x_train,     y_train,     "train_set.txt")

### Salvando o dataframe completo 

Salva dataframe contendo anotacoes de entidades, trechos de texto contendo as anotacoes e as labels (IOB) para cada palavra no texto.

In [25]:
data.to_csv('labeled.csv', index=False)